# Preprocess raw EDF data to filtered FIF format

Define event names in a dictionary

In [1]:
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt

from mne.preprocessing import ICA
from pyprep.find_noisy_channels import NoisyChannels

from IPython import display

import eeglabio

Find EDF file based on subject number and load using MNE

In [2]:
data_dir ="../../Data/EEG DATA/"
save_dir ="../../Data/Processed Data/"

In [3]:
sub_num = 'C27' # Set this at the start for each subject name

In [16]:
sub_id=''; acq_id=''
for folder in os.listdir(data_dir):
    save_fname_fif = sub_num[:3] + '_preprocessed-raw.fif'
    if folder.startswith(sub_num) and save_fname_fif not in os.listdir(save_dir):
        sub_id=folder
        print(sub_id)
        print(save_fname_fif)

for subfile in os.listdir(os.path.join(data_dir,sub_id)):
    if not subfile.endswith(".edf"):
        continue
    else:
        acq_id = subfile;

# read data, set EOG channel, and drop unused channels
print(f"{sub_id}\nreading raw file...")
eeg_data_raw_file = os.path.join(data_dir,sub_id,acq_id)

raw = mne.io.read_raw_edf(eeg_data_raw_file)

wrong_64_mtg_flag = 0
if "FT7" in raw.ch_names:
    wrong_64_mtg_flag = 1
    eog_adj = 4
else:
    eog_adj = 5

Fp1_eog_flag = 0
if "VEO" in raw.ch_names:
    del raw
    raw = mne.io.read_raw_edf(eeg_data_raw_file, eog=["VEO"], preload=True)
elif "VEOG" in raw.ch_names:
    del raw
    raw = mne.io.read_raw_edf(eeg_data_raw_file, eog=["VEOG"], preload=True)
elif "X" in raw.ch_names:
    Fp1_eog_flag = 1
    del raw
    raw = mne.io.read_raw_edf(eeg_data_raw_file, eog=["Fp1"], preload=True)

# display.clear_output(wait=True)

# high level inspection
print(raw.ch_names)
print(len(raw.ch_names))

# drop non EEG channels
if "HEOG" in raw.ch_names:
    non_eeg_chs = ['HEOG', 'EKG', 'EMG', 'Trigger']
elif "HEO" in raw.ch_names:
    non_eeg_chs = ['HEO', 'EKG', 'EMG', 'Trigger']
elif "X" in raw.ch_names and "Oth4" in raw.ch_names:
    non_eeg_chs = ['X', 'Y', 'Z', 'Oth4']
elif "X" in raw.ch_names:
    non_eeg_chs = ['X', 'Y', 'Z']
elif "EEG66" in raw.ch_names:
    non_eeg_chs = ['EEG66','EEG67','EEG68','EEG69']
raw.drop_channels(non_eeg_chs)

# make adjustment for wrong montage subjects
if wrong_64_mtg_flag:
    raw.drop_channels(['FT7','FT8','PO5','PO6']) # for C24, 055, 056, 047

print(raw.ch_names)
print(len(raw.ch_names))

# load electrode locations 
print(f"{sub_id}\nsetting custom montage...")
if wrong_64_mtg_flag:
    custom_montage = mne.channels.read_custom_montage('../Misc/Montage/Hydro_Neo_Net_64_xyz_cms_No_FID_Caps.sfp') 
else:
    custom_montage = mne.channels.read_custom_montage('../Misc/Montage/Hydro_Neo_Net_64_xyz_cms_No_FID.sfp') 
raw.set_montage(custom_montage)
# display.clear_output(wait=True)

# apply notch filter
print(f"{sub_id}\napplying notch filter...")
raw = raw.notch_filter(60., notch_widths=10)
display.clear_output(wait=True)

# apply bandpass filter
print(f"{sub_id}\napplying bandpass filter...")
raw = raw.filter(l_freq=1., h_freq=100.)
display.clear_output(wait=True)

# resample data to decrease file size
resample_freq = 400
print(f"{sub_id}\nresampling data from {raw.info['sfreq']} Hz to {resample_freq} Hz...")
raw.resample(resample_freq, npad='auto')
display.clear_output(wait=True)

# find bad channels automatically
print(f"{sub_id}\nremoving bad channels...")
raw_pyprep = NoisyChannels(raw, random_state=42)
raw_pyprep.find_all_bads(ransac=True, channel_wise=False, max_chunk_size=None)
raw.info['bads'] = raw_pyprep.get_bads()
raw.interpolate_bads(reset_bads=True)
display.clear_output(wait=True)

# re-reference channels
print(f"{sub_id}\nre-referencing channels to average...")
raw, _ = mne.set_eeg_reference(raw, ref_channels='average', copy=True)
display.clear_output(wait=True)

# fit ICA 
print(f"{sub_id}\nfitting ICA...")
num_goods = len(raw.ch_names) - len(raw.info['bads']) - 1 # adjust for EOG
ica = ICA(n_components=int(np.floor(num_goods/2)), random_state=42, max_iter='auto')
ica.fit(raw)
display.clear_output(wait=True)

# find EOG artifacts
print(f"{sub_id}\nfinding EOG artifacts...")

# ica.find_bads_eog is BROKEN. alt: exclude first two components
# eog_indices, eog_scores = ica.find_bads_eog(raw,
#                                             threshold='auto')
# ica.exclude = eog_indices
ica.exclude = [0,1]

display.clear_output(wait=True)

# apply ICA
print(f"{sub_id}\napplying ICA...")
ica.apply(raw)
display.clear_output(wait=True)

# save copy of data
print(f"Saving processed data as '{save_fname_fif}'...")

if "VEO" in raw.ch_names:
    raw.drop_channels("VEO")
elif "VEOG" in raw.ch_names:
    raw.drop_channels("VEOG")
elif Fp1_eog_flag:
    raw.set_channel_types({'Fp1': 'eeg'})

# set_dir = '../../SET Data/'
raw.save(save_dir+save_fname_fif, 
         verbose=True, overwrite=True)
# mne.export.export_raw(set_dir+save_fname_fif[:-4]+'.set', 
#                       raw, overwrite=True, verbose=True)
display.clear_output(wait=True)

# high level inspection
print(raw.ch_names)
print('\nNumber of remaining channels: ',len(raw.ch_names) - len(raw.info['bads']))
print('\nDropped channels: ',raw.info['bads'])

print("Raw data preprocessing complete.")

display.clear_output(wait=True)

['Fp1', 'Fpz', 'Fp2', 'AF3', 'AF4', 'F11', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'F12', 'FT11', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT12', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'M1', 'M2', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Cb1', 'Cb2']

Number of remaining channels:  64

Dropped channels:  []
Raw data preprocessing complete.


In [ ]:
# high level inspection
print(raw.ch_names)
print('\nNumber of remaining channels: ',len(raw.ch_names) - len(raw.info['bads']))
print('\nDropped channels: ',raw.info['bads'])

print("Raw data preprocessing complete.")

display.clear_output(wait=True)